<a href="https://colab.research.google.com/github/emello23/Alura-Gemini/blob/main/Agente_nova_saida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pandas google-api-python-client google-generativeai


In [5]:
import pandas as pd
import time
from IPython.display import display, Markdown
from tabulate import tabulate
from urllib.parse import quote_plus
import google.generativeai as genai # SDK do Gemini
import json # Para parsear a resposta do Gemini
from google.colab import userdata # Para acessar a API key
from datetime import datetime, timedelta # Importar datetime e timedelta

# --- Configuração do Agente e API Key ---
GOOGLE_API_KEY = None
gemini_model_instance = None # Renomeado para clareza

try:
    # O nome padrão para secrets é sem o sufixo _, a menos que você tenha nomeado especificamente assim.
    # Vou usar 'GOOGLE_API_KEY' como padrão, conforme a prática comum.
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        display(Markdown("<font color='red'>**Erro Crítico: GOOGLE_API_KEY não encontrada nos Secrets do Colab.** "
                         "Por favor, configure-a com o nome 'GOOGLE_API_KEY'.</font>"))
    else:
        genai.configure(api_key=GOOGLE_API_KEY)
        gemini_model_instance = genai.GenerativeModel(
            model_name='gemini-1.5-flash-latest', # Modelo eficiente e capaz
            # A opção response_mime_type pode ser usada com modelos mais recentes
            # e configurações de segurança apropriadas.
            # generation_config=genai.types.GenerationConfig(
            #     response_mime_type="application/json"
            # )
        )
        display(Markdown("✅ *SDK do Gemini e modelo (`gemini-1.5-flash-latest`) configurados com sucesso.*"))
except Exception as e:
    display(Markdown(f"<font color='red'>**Erro Crítico ao configurar o SDK do Gemini:** {e}. "
                     "Verifique sua API Key, permissões e se o nome do secret está correto ('GOOGLE_API_APIKEY'). " # Corrigido para 'GOOGLE_API_KEY'
                     "O agente não funcionará sem isso.</font>"))
    # O script poderia parar aqui ou continuar com funcionalidades limitadas se houvesse um fallback.
    # Para este agente, o Gemini é essencial.

# --- Funções Utilitárias ---

def carregar_dados_viagem(caminho_arquivo: str) -> pd.DataFrame | None:
    """
    Carrega os dados de viagem do arquivo CSV e valida as colunas necessárias.
    """
    colunas_requeridas = ['cidade', 'pais', 'data_chegada', 'data_partida', 'hospedagem']
    try:
        df = pd.read_csv(caminho_arquivo)

        # Validar se as colunas requeridas existem
        if not all(col in df.columns for col in colunas_requeridas):
            colunas_faltando = [col for col in colunas_requeridas if col not in df.columns]
            display(Markdown(f"<font color='red'>**Erro: O arquivo `{caminho_arquivo}` não contém as colunas requeridas.** "
                             f"Faltam as seguintes colunas: {', '.join(colunas_faltando)}.</font>"))
            return None

        display(Markdown(f"### Arquivo de Viagem (`{caminho_arquivo}`)"))
        display(df[colunas_requeridas]) # Exibe apenas as colunas relevantes
        return df
    except FileNotFoundError:
        display(Markdown(f"<font color='red'>**Erro: Arquivo `{caminho_arquivo}` não encontrado.** Verifique o nome e o local.</font>"))
        return None
    except Exception as e:
        display(Markdown(f"<font color='red'>**Erro ao carregar o arquivo CSV `{caminho_arquivo}`:** {e}</font>"))
        return None

def gerar_links_pesquisa_google(cidade: str, pais: str) -> tuple[str, str, str]:
    """Gera links de pesquisa úteis para Google Search e Google Maps."""
    query_atracoes = f"principais atrações turísticas em {cidade} {pais}"
    link_google_search = f"https://www.google.com/search?q={quote_plus(query_atracoes)}"
    query_maps_atracoes = f"atrações turísticas em {cidade}, {pais}"
    link_google_maps_atracoes = f"https://www.google.com/maps/search/{quote_plus(query_maps_atracoes)}"
    query_maps_cidade = f"{cidade}, {pais}"
    link_google_maps_cidade = f"https://www.google.com/maps/place/{quote_plus(query_maps_cidade)}"
    return link_google_search, link_google_maps_atracoes, link_google_maps_cidade

# --- Definição do Agente Pesquisador de Atrações ---

class AgentePesquisadorAtracoes:
    """
    Um agente responsável por pesquisar atrações turísticas usando o Gemini
    e fornecer links úteis para pesquisa manual.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def _obter_sugestoes_gemini(self, cidade: str, pais: str, top_n: int = 7) -> list[dict]:
        """
        Ferramenta interna do agente para consultar o Gemini sobre atrações.
        """
        display(Markdown(f"🧠 *Agente consultando Gemini sobre atrações em: **{cidade}, {pais}** (Top {top_n})...*"))

        prompt = f"""
        Você é um assistente de planejamento de viagens altamente especializado e eficiente.
        Sua tarefa é identificar as {top_n} principais e mais recomendadas atrações turísticas para a cidade de {cidade}, localizada em {pais}.

        Critérios para sua seleção:
        1.  **Popularidade e Reconhecimento:** Atrações bem conhecidas e frequentemente visitadas.
        2.  **Qualidade das Avaliações:** Lugares geralmente com avaliações positivas (ex: acima de 4.0/5 estrelas).
        3.  **Relevância Cultural/Histórica:** Locais com significado importante.
        4.  **Diversidade de Experiências:** Inclua uma mistura de tipos, como museus, monumentos, parques, mercados, igrejas/catedrais, mirantes, etc., quando apropriado para a cidade.
        5.  **Singularidade:** Atrações que oferecem uma experiência única ou icônica da cidade/região.

        Formato da Resposta:
        Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando uma atração,
        contendo EXATAMENTE as seguintes chaves:
        - "nome": (string) O nome oficial e completo da atração.
        - "tipo_principal": (string) A categoria principal da atração (ex: "Museu de Arte", "Catedral Gótica", "Parque Urbano", "Mercado Histórico", "Monumento Nacional", "Mirante Panorâmico"). Seja específico.
        - "descricao_enxuta": (string) Uma descrição concisa e informativa (1-2 frases) que justifique sua inclusão, destacando seus principais atrativos.
        - "avaliacao_geral": (string, opcional) Uma estimativa da avaliação geral se for amplamente conhecida (ex: "Excelente", "Muito Popular", "4.5/5 estrelas"). Use "N/A" se não houver uma estimativa clara.
        - "destaque_principal": (string) O motivo principal pelo qual um turista deveria visitar (ex: "Vistas incríveis da cidade", "Coleção de arte renascentista", "Arquitetura impressionante", "Atmosfera vibrante").

        Exemplo de um item na lista JSON:
        {{
          "nome": "Museu do Louvre",
          "tipo_principal": "Museu de Arte e Antiguidades",
          "descricao_enxuta": "Um dos maiores e mais visitados museus do mundo, lar de obras-primas como a Mona Lisa e a Vênus de Milo.",
          "avaliacao_geral": "4.7/5 estrelas",
          "destaque_principal": "Coleção de arte de renome mundial"
        }}

        Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.4, # Um pouco mais factual, menos aleatório
                    max_output_tokens=3072 # Espaço suficiente para a lista JSON
                ),
                safety_settings=self.safety_settings
            )

            json_text = response.text.strip()
            # Tentativa robusta de extrair JSON, mesmo que o modelo adicione ```json ... ```
            if json_text.startswith("```json"):
                json_text = json_text[7:]
            if json_text.endswith("```"):
                json_text = json_text[:-3]
            json_text = json_text.strip()

            if not json_text:
                display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia para {cidade}.</font>"))
                return []

            atracoes = json.loads(json_text)
            # Garantir que é uma lista, mesmo que o Gemini retorne algo diferente
            if not isinstance(atracoes, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para {cidade}.</font>"))
                 return []

            display(Markdown(f"✅ *Gemini retornou {len(atracoes)} sugestões de atrações para {cidade}.*"))
            return atracoes[:top_n]
        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return []
        except Exception as e:
            # Captura de erros mais genéricos, como problemas de API (quota, etc.)
            # ou bloqueios de segurança não esperados.
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return []

    def pesquisar_destino(self, cidade: str, pais: str, top_n_sugestoes: int = 7) -> dict:
        """
        Executa a pesquisa de atrações para um destino específico.
        """
        display(Markdown(f"\n### 🌍 Agente Iniciando Pesquisa para: **{cidade}, {pais}**"))

        links = gerar_links_pesquisa_google(cidade, pais)
        sugestoes_gemini = self._obter_sugestoes_gemini(cidade, pais, top_n=top_n_sugestoes)

        display(Markdown(f"--- Pesquisa para **{cidade}, {pais}** finalizada pelo agente. ---"))
        return {
            "cidade": cidade,
            "pais": pais,
            "link_google_search": links[0],
            "link_google_maps_atracoes": links[1],
            "link_google_maps_cidade": links[2],
            "sugestoes_gemini": sugestoes_gemini
        }

# --- Função Principal de Execução ---
def executar_planejador_viagem(caminho_arquivo_csv: str = 'europa.csv'):
    """
    Orquestra o carregamento dos dados da viagem e a pesquisa de atrações para cada destino.
    Recebe o caminho do arquivo CSV como parâmetro.
    """
    if not GOOGLE_API_KEY or not gemini_model_instance:
        display(Markdown("<font color='red'>**Execução Interrompida.** API Key do Google ou modelo Gemini não configurados corretamente na inicialização.</font>"))
        return None # Retorna None se a configuração falhar

    df_viagem = carregar_dados_viagem(caminho_arquivo_csv)

    if df_viagem is None:
        display(Markdown("<font color='red'>**Não foi possível prosseguir sem os dados da viagem.**</font>"))
        return None # Retorna None se o carregamento falhar

    agente_pesquisador = AgentePesquisadorAtracoes(modelo_llm=gemini_model_instance)
    # agente_montador = MontadorDeRoteiros(modelo_llm=gemini_model_instance) # Instanciado dentro do loop se necessário

    dados_compilados = {
        "titulo_pesquisa": "Relatório de Pesquisa de Atrações do Agente",
        "data_execucao_pesquisa": time.strftime("%Y-%m-%d %H:%M:%S"),
        "data_geracao_relatorio_utc": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC"),
        "data_geracao_relatorio_brasilia": (datetime.utcnow() - timedelta(hours=3)).strftime("%Y-%m-%d %H:%M:%S BRT"),
        "destinos_pesquisados": []
    }


    display(Markdown("\n# 🗺️ **Relatório de Pesquisa de Atrações do Agente** ✈️"))

    resultados_completos = []
    for indice, linha_viagem in df_viagem.iterrows():
        cidade = linha_viagem['cidade']
        pais = linha_viagem['pais']

        resultado_destino = agente_pesquisador.pesquisar_destino(cidade, pais, top_n_sugestoes=15)

        # Adicionar dados da viagem original ao resultado
        resultado_destino['data_chegada'] = linha_viagem['data_chegada']
        resultado_destino['data_partida'] = linha_viagem['data_partida']
        resultado_destino['hospedagem'] = linha_viagem['hospedagem']
        resultados_completos.append(resultado_destino)

        # Exibição imediata por destino
        display(Markdown(f"\n## 📍 Destino: **{resultado_destino['cidade']}, {resultado_destino['pais']}**"))
        display(Markdown(f"🗓️ **Período:** {resultado_destino['data_chegada']} a {resultado_destino['data_partida']}"))
        display(Markdown(f"🏨 **Hospedagem:** {resultado_destino['hospedagem']}"))

        display(Markdown("#### 🔍 Links Úteis para Pesquisa Manual Detalhada (abrem em nova aba):"))
        display(Markdown(f"- **[Google Search: Principais atrações]({resultado_destino['link_google_search']})** "))
        display(Markdown(f"- **[Google Maps: Atrações na área]({resultado_destino['link_google_maps_atracoes']})** "))
        display(Markdown(f"- **[Google Maps: Visão geral da cidade]({resultado_destino['link_google_maps_cidade']})** "))

        if resultado_destino['sugestoes_gemini']:
            display(Markdown("#### ✨ Sugestões de Atrações Principais (via Agente Gemini):"))

            tabela_atracoes_data = []
            # Chaves como definidas no prompt do Gemini
            headers = ["Nº", "Atração", "Tipo Principal", "Descrição Enxuta", "Avaliação Geral", "Destaque Principal"]

            for i, atracao in enumerate(resultado_destino['sugestoes_gemini']):
                tabela_atracoes_data.append([
                    i + 1,
                    atracao.get('nome', 'N/A'),
                    atracao.get('tipo_principal', 'N/A'),
                    atracao.get('descricao_enxuta', 'N/A'),
                    atracao.get('avaliacao_geral', 'N/A'),
                    atracao.get('destaque_principal', 'N/A')
                ])

            tabela_formatada = tabulate(tabela_atracoes_data, headers=headers, tablefmt="pipe", stralign="left")
            display(Markdown(tabela_formatada))
        else:
            display(Markdown("<font color='orange'>⚠️ Nenhuma sugestão de atração específica retornada pelo Gemini para este destino. "
                             "Utilize os links de pesquisa manual acima.</font>"))
        display(Markdown("---")) # Separador visual entre destinos

    dados_compilados["destinos_pesquisados"] = resultados_completos

    # Salva todos os dados de atrações em um arquivo JSON
    output_filename = "atracoes_pesquisadas.json"
    try:
        with open(output_filename, "w", encoding="utf-8") as f:
            json.dump(dados_compilados, f, ensure_ascii=False, indent=4)
        print(f"\nDados completos da pesquisa salvos em `{output_filename}`.")
    except Exception as e:
        print(f"\nErro ao salvar dados em JSON: {e}")

    return dados_compilados

# --- Ponto de Entrada da Execução ---
if __name__ == "__main__":
    # Certifique-se de que as bibliotecas estão instaladas
    # Em um notebook Colab, você executaria !pip install -q pandas google-generativeai tabulate em uma célula separada.
    # Agora você pode chamar a função passando o caminho do seu arquivo CSV, por exemplo:
    # executar_planejador_viagem('meu_arquivo_viagem.csv')
    # Se nenhum caminho for fornecido, ele usará 'europa.csv' por padrão.
    dados_finais_viagem = executar_planejador_viagem('/content/europa.csv')

✅ *SDK do Gemini e modelo (`gemini-1.5-flash-latest`) configurados com sucesso.*

### Arquivo de Viagem (`/content/europa.csv`)

,cidade,pais,data_chegada,data_partida,hospedagem
0,Roma,Itália,2025-09-13,2025-09-16,Via Francesco Caracciolo n.7
1,Florença,Itália,2025-09-16,2025-09-19,Hotel Bodoni
2,Viena,Aústria,2025-09-20,2025-09-24,Hilton Vienna Park



# 🗺️ **Relatório de Pesquisa de Atrações do Agente** ✈️


### 🌍 Agente Iniciando Pesquisa para: **Roma, Itália**

...

- **[Google Maps: Visão geral da cidade](https://www.google.com/maps/place/Viena%2C+A%C3%BAstria)** 

#### ✨ Sugestões de Atrações Principais (via Agente Gemini):

|   Nº | Atração                       | Tipo Principal                   | Descrição Enxuta                                                                                                                                  | Avaliação Geral   | Destaque Principal                                 |
|-----:|:------------------------------|:---------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------|:------------------|:---------------------------------------------------|
|    1 | Palácio de Schönbrunn         | Palácio Histórico e Jardins      | Residência de verão dos Habsburgos, com exuberantes jardins, palácios e museus, oferecendo um vislumbre da história imperial austríaca.           | Excelente         | Magnificência arquitetônica e paisagística         |
|    2 | Hofburg                       | Complexo de Palácios Imperiais   | Antiga residência de inverno dos Habsburgos, abrigando museus, capelas e a Escola Espanhola de Equitação, um centro histórico e cultural.         | Muito Popular     | História imperial e arquitetura deslumbrante       |
|    3 | Catedral de Santo Estêvão     | Catedral Gótica                  | Símbolo icônico de Viena, com arquitetura gótica impressionante, tesouros religiosos e vistas panorâmicas do topo da torre sul.                   | Excelente         | Arquitetura gótica e importância histórica         |
|    4 | Museu de História da Arte     | Museu de Arte                    | Coleção excepcional de arte mundial, desde a antiguidade até o século XVIII, com obras-primas de artistas renomados como Rafael e Michelangelo.   | 4.5/5 estrelas    | Coleção de arte abrangente e de alta qualidade     |
|    5 | Belvedere                     | Palácio e Museu de Arte          | Conjunto de dois palácios barrocos que abrigam a maior coleção de obras de Gustav Klimt, incluindo 'O Beijo'.                                     | Excelente         | Obras-primas de Gustav Klimt e arquitetura barroca |
|    6 | Prater                        | Parque de Diversões Histórico    | Parque de diversões icônico com a famosa Roda-Gigante, oferecendo entretenimento e vistas panorâmicas da cidade.                                  | Muito Popular     | Entretenimento e vistas da cidade                  |
|    7 | Naschmarkt                    | Mercado Histórico                | Um dos maiores e mais antigos mercados ao ar livre da Europa, oferecendo uma variedade de produtos frescos, especiarias e comidas internacionais. | 4.0/5 estrelas    | Experiência gastronômica e cultural vibrante       |
|    8 | Palácio de Belvedere Superior | Museu de Arte Barroca            | Palácio barroco que abriga uma coleção impressionante de arte austríaca, com destaque para as obras de Gustav Klimt.                              | Excelente         | Obras-primas de Gustav Klimt e arquitetura barroca |
|    9 | Casa da Música                | Centro de Música Moderna         | Centro de concertos de arquitetura moderna e inovadora, com acústica excepcional e uma programação diversificada.                                 | 4.5/5 estrelas    | Arquitetura moderna e concertos de alta qualidade  |
|   10 | Stephansplatz                 | Praça Histórica                  | Praça central de Viena, localizada em frente à Catedral de Santo Estêvão, com atmosfera vibrante e edifícios históricos.                          | Muito Popular     | Atmosfera histórica e central                      |
|   11 | Roda-Gigante do Prater        | Atração de Entretenimento        | Uma das mais antigas e famosas rodas-gigantes do mundo, oferecendo vistas panorâmicas de Viena do alto.                                           | Muito Popular     | Vistas panorâmicas de Viena                        |
|   12 | Volksgarten                   | Parque Urbano                    | Parque público com jardins bem cuidados, monumentos históricos e o famoso monumento a Mozart.                                                     | 4.0/5 estrelas    | Espaço verde tranquilo no coração da cidade        |
|   13 | Museu Albertina               | Museu de Artes Gráficas          | Coleção excepcional de gravuras, desenhos e fotografias, com obras de artistas renomados como Michelangelo e Picasso.                             | 4.5/5 estrelas    | Coleção de artes gráficas de renome mundial        |
|   14 | Danúbio                       | Rio e Paisagem Urbana            | O rio Danúbio atravessa Viena, oferecendo passeios de barco, vistas pitorescas e oportunidades para relaxar nas margens.                          | N/A               | Paisagens urbanas e passeios de barco              |
|   15 | Palácio do Arsenal            | Museu de Arte Militar e História | Museu que apresenta a história militar da Áustria, com uma coleção impressionante de armas, armaduras e artefatos históricos.                     | 4.0/5 estrelas    | História militar austríaca e artefatos históricos  |

---


Dados completos da pesquisa salvos em `atracoes_pesquisadas.json`.


In [9]:

# --- Definição do Agente Montador de Roteiros ---

class MontadorDeRoteiros:
    """
    Um agente responsável por organizar as atrações por dia, considerando proximidade,
    hospedagem como ponto de partida/chegada, e adicionando elementos para tornar o roteiro prazeroso e divertido.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def montar_roteiro(self, cidade: str, pais: str, atracoes: list, hospedagem: str, data_chegada: str, data_partida: str) -> dict:
        """
        Monta um roteiro diário otimizado para as atrações, considerando a hospedagem e datas.
        """
        from datetime import datetime
        delta = datetime.strptime(data_partida, "%Y-%m-%d") - datetime.strptime(data_chegada, "%Y-%m-%d")
        dias_viagem = delta.days + 1 # Inclui o dia de chegada e partida

        atracoes_str = json.dumps(atracoes, ensure_ascii=False, indent=2)

        prompt = f"""
        Você é um especialista em planejamento de roteiros de viagem, focado em otimização e experiência do viajante.
        Sua tarefa é criar um roteiro diário detalhado para uma viagem a {cidade}, {pais}, com base nas atrações fornecidas.
        A viagem será de {data_chegada} a {data_partida}, totalizando {dias_viagem} dias.
        A hospedagem principal é em: {hospedagem}.

        Atrações disponíveis (formato JSON):
        ```json
        {atracoes_str}
        ```

        Critérios para o roteiro:
        1.  **Otimização por Proximidade:** Agrupe atrações próximas geograficamente para minimizar deslocamentos.
        2.  **Ponto de Partida/Chegada:** Comece o dia próximo à hospedagem e, idealmente, termine o dia retornando a ela.
        3.  **Fluxo Lógico:** Organize as visitas de forma sequencial e natural (ex: museus pela manhã, parques à tarde, jantar à noite).
        4.  **Experiência do Viajante:** Inclua sugestões de refeições (café, almoço, jantar), pausas, transporte e atividades opcionais para tornar o dia prazeroso e divertido.
        5.  **Informações Essenciais:** Para cada atração, inclua o nome, tipo, e um breve lembrete do destaque principal.
        6.  **Número de Atrações por Dia:** Distribua as atrações de forma realista, evitando sobrecarga. Considere 3-5 atrações principais por dia, dependendo da complexidade.
        7.  **Flexibilidade:** Adicione uma nota sobre a flexibilidade do roteiro e a possibilidade de ajustes.

        Formato da Resposta:
        Retorne **estritamente um objeto JSON**. O objeto deve conter uma chave "roteiro_diario" que é uma lista de objetos,
        onde cada objeto representa um dia e contém as seguintes chaves:
        - "dia": (inteiro) O número do dia (ex: 1, 2, 3...).
        - "data": (string) A data correspondente ao dia (formato YYYY-MM-DD).
        - "atividades": (lista de strings) Uma lista de atividades para o dia, incluindo atrações, refeições, transporte, etc.

        Exemplo de formato de saída:
        {{
          "roteiro_diario": [
            {{
              "dia": 1,
              "data": "2025-07-01",
              "atividades": [
                "Manhã: Café da manhã na hospedagem.",
                "Manhã: Visita ao Museu do Louvre (Coleção de arte de renome mundial). Sugestão: Chegar cedo para evitar filas.",
                "Almoço: Sugestão de bistrô próximo ao museu.",
                "Tarde: Passeio pela Torre Eiffel (Vistas panorâmicas de Paris). Sugestão: Comprar ingressos online com antecedência.",
                "Noite: Jantar em restaurante com vista para o Sena. Retorno à hospedagem."
              ]
            }},
            {{
              "dia": 2,
              "data": "2025-07-02",
              "atividades": [
                "Manhã: Café da manhã.",
                "Manhã: Visita à Catedral de Notre-Dame (Arquitetura gótica impressionante). Sugestão: Verificar horários de missa.",
                "Almoço: Piquenique no Jardim de Luxemburgo.",
                "Tarde: Exploração do bairro de Montmartre e Sacré-Cœur (Charme artístico e vistas elevadas). Sugestão: Usar transporte público para subir a colina.",
                "Noite: Jantar e show de cabaré. Retorno à hospedagem."
              ]
            }}
          ]
        }}

        Garanta que a saída seja SOMENTE o objeto JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7, # Um pouco mais criativo para roteiros
                    max_output_tokens=4096 # Espaço suficiente para o roteiro detalhado
                ),
                safety_settings=self.safety_settings
            )

            json_text = response.text.strip()
            if json_text.startswith("```json"):
                json_text = json_text[7:]
            if json_text.endswith("```"):
                json_text = json_text[:-3]
            json_text = json_text.strip()

            if not json_text:
                display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia para o roteiro de {cidade}.</font>"))
                return {"roteiro_diario": []}

            roteiro = json.loads(json_text)
            if not isinstance(roteiro, dict) or "roteiro_diario" not in roteiro:
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não um dicionário com 'roteiro_diario') para {cidade}.</font>"))
                 return {"roteiro_diario": []}

            display(Markdown(f"✅ *Gemini montou um roteiro com {len(roteiro.get('roteiro_diario', []))} dias para {cidade}.*"))
            return roteiro
        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para o roteiro de {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return {"roteiro_diario": []}
        except Exception as e:
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para o roteiro de {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return {"roteiro_diario": []}




In [7]:

# --- Definição do Agente Montador de Roteiros ---

class MontadorDeRoteiros:
    """
    Um agente responsável por organizar as atrações por dia, considerando proximidade,
    hospedagem como ponto de partida/chegada, e adicionando elementos para tornar o roteiro prazeroso e divertido.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def montar_roteiro(self, cidade: str, pais: str, atracoes: list, hospedagem: str, data_chegada: str, data_partida: str) -> dict:
        """
        Monta um roteiro diário otimizado para as atrações, considerando a hospedagem e datas.
        """
        from datetime import datetime
        delta = datetime.strptime(data_partida, "%Y-%m-%d") - datetime.strptime(data_chegada, "%Y-%m-%d")
        dias_viagem = delta.days + 1 # Inclui o dia de chegada e partida

        atracoes_str = json.dumps(atracoes, ensure_ascii=False, indent=2)

        prompt = f"""
        Você é um especialista em planejamento de roteiros de viagem, focado em otimização e experiência do viajante.
        Sua tarefa é criar um roteiro diário detalhado para uma viagem a {cidade}, {pais}, com base nas atrações fornecidas.
        A viagem será de {data_chegada} a {data_partida}, totalizando {dias_viagem} dias.
        A hospedagem principal é em: {hospedagem}.

        Atrações disponíveis (formato JSON):
        ```json
        {atracoes_str}
        ```

        Critérios para o roteiro:
        1.  **Otimização por Proximidade:** Agrupe atrações próximas geograficamente para minimizar deslocamentos.
        2.  **Ponto de Partida/Chegada:** Comece o dia próximo à hospedagem e, idealmente, termine o dia retornando a ela.
        3.  **Fluxo Lógico:** Organize as visitas de forma sequencial e natural (ex: museus pela manhã, parques à tarde, jantar à noite).
        4.  **Experiência do Viajante:** Inclua sugestões de refeições (café, almoço, jantar), pausas, transporte e atividades opcionais para tornar o dia prazeroso e divertido.
        5.  **Informações Essenciais:** Para cada atração, inclua o nome, tipo, e um breve lembrete do destaque principal.
        6.  **Número de Atrações por Dia:** Distribua as atrações de forma realista, evitando sobrecarga. Considere 3-5 atrações principais por dia, dependendo da complexidade.
        7.  **Flexibilidade:** Adicione uma nota sobre a flexibilidade do roteiro e a possibilidade de ajustes.

        Formato da Resposta:
        Retorne **estritamente um objeto JSON**. O objeto deve conter uma chave "roteiro_diario" que é uma lista de objetos,
        onde cada objeto representa um dia e contém as seguintes chaves:
        - "dia": (inteiro) O número do dia (ex: 1, 2, 3...).
        - "data": (string) A data correspondente ao dia (formato YYYY-MM-DD).
        - "atividades": (lista de strings) Uma lista de atividades para o dia, incluindo atrações, refeições, transporte, etc.

        Exemplo de formato de saída:
        {{
          "roteiro_diario": [
            {{
              "dia": 1,
              "data": "2025-07-01",
              "atividades": [
                "Manhã: Café da manhã na hospedagem.",
                "Manhã: Visita ao Museu do Louvre (Coleção de arte de renome mundial). Sugestão: Chegar cedo para evitar filas.",
                "Almoço: Sugestão de bistrô próximo ao museu.",
                "Tarde: Passeio pela Torre Eiffel (Vistas panorâmicas de Paris). Sugestão: Comprar ingressos online com antecedência.",
                "Noite: Jantar em restaurante com vista para o Sena. Retorno à hospedagem."
              ]
            }},
            {{
              "dia": 2,
              "data": "2025-07-02",
              "atividades": [
                "Manhã: Café da manhã.",
                "Manhã: Visita à Catedral de Notre-Dame (Arquitetura gótica impressionante). Sugestão: Verificar horários de missa.",
                "Almoço: Piquenique no Jardim de Luxemburgo.",
                "Tarde: Exploração do bairro de Montmartre e Sacré-Cœur (Charme artístico e vistas elevadas). Sugestão: Usar transporte público para subir a colina.",
                "Noite: Jantar e show de cabaré. Retorno à hospedagem."
              ]
            }}
          ]
        }}

        Garanta que a saída seja SOMENTE o objeto JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7, # Um pouco mais criativo para roteiros
                    max_output_tokens=4096 # Espaço suficiente para o roteiro detalhado
                ),
                safety_settings=self.safety_settings
            )

            json_text = response.text.strip()
            if json_text.startswith("```json"):
                json_text = json_text[7:]
            if json_text.endswith("```"):
                json_text = json_text[:-3]
            json_text = json_text.strip()

            if not json_text:
                display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia para o roteiro de {cidade}.</font>"))
                return {"roteiro_diario": []}

            roteiro = json.loads(json_text)
            if not isinstance(roteiro, dict) or "roteiro_diario" not in roteiro:
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não um dicionário com 'roteiro_diario') para {cidade}.</font>"))
                 return {"roteiro_diario": []}

            display(Markdown(f"✅ *Gemini montou um roteiro com {len(roteiro.get('roteiro_diario', []))} dias para {cidade}.*"))
            return roteiro
        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para o roteiro de {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return {"roteiro_diario": []}
        except Exception as e:
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para o roteiro de {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return {"roteiro_diario": []}




In [8]:
dados_compilados = executar_planejador_viagem("/content/europa.csv")

### Arquivo de Viagem (`/content/europa.csv`)

,cidade,pais,data_chegada,data_partida,hospedagem
0,Roma,Itália,2025-09-13,2025-09-16,Via Francesco Caracciolo n.7
1,Florença,Itália,2025-09-16,2025-09-19,Hotel Bodoni
2,Viena,Aústria,2025-09-20,2025-09-24,Hilton Vienna Park



# 🗺️ **Relatório de Pesquisa de Atrações do Agente** ✈️


### 🌍 Agente Iniciando Pesquisa para: **Roma, Itália**

🧠 *Agente consultando Gemini sobre atrações em: **Roma, Itália** (Top 15)...*

...

- **[Google Maps: Visão geral da cidade](https://www.google.com/maps/place/Viena%2C+A%C3%BAstria)** 

#### ✨ Sugestões de Atrações Principais (via Agente Gemini):

|   Nº | Atração                       | Tipo Principal                 | Descrição Enxuta                                                                                                                                  | Avaliação Geral   | Destaque Principal                                   |
|-----:|:------------------------------|:-------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------|:------------------|:-----------------------------------------------------|
|    1 | Palácio de Schönbrunn         | Palácio Histórico e Jardins    | Residência de verão dos Habsburgos, com exuberantes jardins, palácios e museus, oferecendo um vislumbre da história imperial austríaca.           | Excelente         | Arquitetura imperial e jardins deslumbrantes         |
|    2 | Hofburg                       | Complexo de Palácios           | Antiga residência de inverno dos Habsburgos, abrigando museus, capelas e apartamentos imperiais, mostrando a opulência da monarquia austríaca.    | Muito Popular     | História imperial e arquitetura grandiosa            |
|    3 | Catedral de São Estêvão       | Catedral Gótica                | Símbolo icônico de Viena, com uma arquitetura gótica impressionante, tesouros religiosos e vistas panorâmicas do topo da torre sul.               | Excelente         | Arquitetura gótica e vistas panorâmicas              |
|    4 | Museu de História da Arte     | Museu de Arte                  | Abriga uma vasta coleção de pinturas, esculturas e objetos de arte de diferentes épocas e culturas, incluindo obras-primas de artistas renomados. | 4.5/5 estrelas    | Coleção de arte abrangente e diversificada           |
|    5 | Belvedere                     | Palácio e Museu de Arte        | Conjunto de palácios barrocos que abriga a maior coleção de obras de Gustav Klimt, incluindo 'O Beijo'.                                           | Excelente         | Obras-primas de Gustav Klimt e arquitetura barroca   |
|    6 | Prater                        | Parque de Diversões            | Parque de diversões histórico e famoso pela icônica roda-gigante, oferecendo entretenimento para todas as idades.                                 | Muito Popular     | Entretenimento e atmosfera festiva                   |
|    7 | Naschmarkt                    | Mercado Histórico              | Um dos maiores e mais antigos mercados ao ar livre da Europa, oferecendo uma variedade de produtos frescos, especiarias e comida internacional.   | 4.0/5 estrelas    | Experiência gastronômica e cultural vibrante         |
|    8 | Palácio de Belvedere Inferior | Museu de Arte                  | Palácio barroco que abriga uma coleção notável de arte austríaca, incluindo obras de Gustav Klimt e Egon Schiele.                                 | Excelente         | Arte austríaca e arquitetura barroca                 |
|    9 | Casa da Música                | Centro Cultural e de Concertos | Moderna sala de concertos com arquitetura impressionante e acústica excepcional, oferecendo uma variedade de concertos e eventos.                 | 4.5/5 estrelas    | Arquitetura moderna e concertos de alta qualidade    |
|   10 | Stephansdom (Torre Norte)     | Mirante Panorâmico             | Suba a torre norte da Catedral de São Estêvão para vistas deslumbrantes da cidade de Viena.                                                       | Excelente         | Vistas panorâmicas de Viena                          |
|   11 | Volksgarten                   | Parque Urbano                  | Belo parque no centro de Viena, com jardins bem cuidados, monumentos e o famoso Templo de Theseus.                                                | Muito Popular     | Espaço verde tranquilo no centro da cidade           |
|   12 | Museu Albertina               | Museu de Arte Gráfica          | Abriga uma coleção excepcional de gravuras, desenhos e fotografias, incluindo obras de mestres como Dürer e Michelangelo.                         | 4.5/5 estrelas    | Coleção de arte gráfica de renome mundial            |
|   13 | Danúbio                       | Rio e Passeios de Barco        | Passeios de barco pelo rio Danúbio oferecem vistas únicas da cidade e seus arredores.                                                             | Muito Popular     | Vistas panorâmicas da cidade a partir do rio         |
|   14 | Palácio do Belvedere Superior | Palácio e Museu de Arte        | Palácio barroco que abriga importantes coleções de arte austríaca, incluindo obras de Gustav Klimt e Egon Schiele.                                | Excelente         | Obras-primas de arte austríaca e arquitetura barroca |
|   15 | MuseumsQuartier               | Complexo de Museus             | Complexo de museus modernos e contemporâneos, incluindo o Leopold Museum e o MUMOK, oferecendo uma ampla gama de exposições.                      | Muito Popular     | Diversidade de museus e arquitetura moderna          |

---


Dados completos da pesquisa salvos em `atracoes_pesquisadas.json`.
